In [7]:
# Ensure that the directory is formatted as follows for parsing purposes:
# cv-valid-train/
# ├── metadata.csv (renamed cv-valid-train.csv: column 'filename' changed to 'file_name')
# └── cv-valid-train/
#     ├── sample-000000.mp3
#     ├── sample-000001.mp3
#     ...
data_dir = '../../../data/common_voice/cv-valid-train' # Path to common_voice/cv-valid-train/ directory

In [11]:
# Load the dataset
from datasets import load_dataset
dataset = load_dataset('audiofolder', data_dir=data_dir, split='train')

Generating train split: 195776 examples [00:18, 10355.52 examples/s]
Generating validation split: 195776 examples [00:19, 10268.49 examples/s]


# Preprocessing

The model was trained on raw audio sampled at 16kHz. Hence the common voice data also should be sampled at 16kHz.

In [12]:
from datasets import Audio

# Cast audio to 16kHz
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

In [13]:
# View a sample
dataset[0]

{'audio': {'path': '/home/lee-j/data/common_voice/cv-valid-train/cv-valid-train/sample-000000.mp3',
  'array': array([-2.91038305e-11, -1.09139364e-11, -1.81898940e-12, ...,
          1.25038903e-04,  7.30113592e-04,  7.36902002e-04]),
  'sampling_rate': 16000},
 'text': 'learn to recognize omens and follow them the old king had said',
 'up_votes': 1,
 'down_votes': 0,
 'age': None,
 'gender': None,
 'accent': None,
 'duration': None}

Wav2Vec2 was trained on audio data only, so for consistency, other data columns are removed.

In [14]:
# Remove the 'up_votes', 'down_votes', 'age', 'gender', 'accent', and 'duration' columns
dataset = dataset.remove_columns(
    ['up_votes', 'down_votes', 'age', 'gender', 'accent', 'duration']
)

In [15]:
# Create train-validation split
split_dataset = dataset.train_test_split(test_size=0.3, seed=42)
split_dataset

DatasetDict({
    train: Dataset({
        features: ['audio', 'text'],
        num_rows: 137043
    })
    test: Dataset({
        features: ['audio', 'text'],
        num_rows: 58733
    })
})

In CTC, it is common to classify speech chunks into letters. Extract all letters from the data and build vocabulary.

In [19]:
# Function to extract characters
def extract_all_chars(batch):
    all_text = " ".join(batch["text"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}

vocabs = split_dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=split_dataset.column_names["train"])

Map:   0%|          | 0/137043 [00:00<?, ? examples/s]

Map: 100%|██████████| 58733/58733 [00:02<00:00, 23117.65 examples/s]


In [20]:
# All distinct letters in the training set
vocab_list = list(set(vocabs["train"]["vocab"][0]))

vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'k': 0,
 "'": 1,
 'd': 2,
 'o': 3,
 'x': 4,
 'c': 5,
 'i': 6,
 ' ': 7,
 'r': 8,
 'l': 9,
 'q': 10,
 's': 11,
 'w': 12,
 'g': 13,
 'f': 14,
 'z': 15,
 'm': 16,
 'b': 17,
 'u': 18,
 'e': 19,
 't': 20,
 'v': 21,
 'p': 22,
 'a': 23,
 'j': 24,
 'n': 25,
 'y': 26,
 'h': 27}

In [21]:
# Add unknown and padding tokens
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
# Check total vocab size
print(len(vocab_dict))

30


In [23]:
# Save the vocab file
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [5]:
# Load the tokenizer class from the vocab file
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token=" ")


# Feature Extraction

For feature extraction, we use the ```Wav2Vec2FeatureExtractor``` provided with ```Wav2Vec2```.

The parameters for the feature extractor class are:

- ```feature_size```: Feature size for ```Wav2Vec2``` is 1, as it was trained on raw speech signal.
- ```sampling_rate```: The sampling rate for the dataset has already been set to 16kHz in preprocessing.
- ```padding_value```: For batching purposes, to pad the end of shorter sequences. This is set to ```0```.
- ```do_normalize```: Boolean to normalise the inputs to zero-mean-unit-variance. For speech models, this is usually set to ```True```.
- ```return_attention_mask```: Boolean to return the attention masks. For ```wav2vec2-large-960h```, this is set to ```False```.


In [6]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

```Wav2Vec2``` provides a processor class to wrap the tokenizer and the feature extractor classes together, after which we just need the ```model``` and ```processor``` objects, just like we did in Task 2.

In [7]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

We then prepare the dateset with the processor class. The processor class will:
1. tokenize the target text
2. normalise the audio input

In [27]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

In [28]:
dataset = split_dataset.map(prepare_dataset, remove_columns=split_dataset.column_names["train"])

Map:   0%|          | 0/137043 [00:00<?, ? examples/s]/home/lee-j/github/supreme-parakeet/.venv/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 58733/58733 [22:58<00:00, 42.60 examples/s]  


In [39]:
# import pickle
# # Step 2: Saving Variables
# file_path = 'dataset.pkl'

# # Open the file in binary mode
# with open(file_path, 'wb') as file:
#     # Serialize and write the variable to the file
#     pickle.dump(dataset, file)

# Finetuning the model

## Data collator
First we set up the data collator, taken from an example provided with the Transformers library: https://github.com/huggingface/transformers/blob/7e61d56a45c19284cfda0cee8995fb552f6b1f4e/examples/pytorch/speech-recognition/run_speech_recognition_ctc.py#L219

In [14]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [15]:
# Initialise the data collator
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

## Evaluation Metric

We use word error rate (WER) for evaluation, as it is widely used in automatic speech recognition. ```compute_wer``` is adapted from the same Transformers example.

In [31]:
from datasets import load_metric

get_wer = load_metric('wer')

/tmp/ipykernel_45394/3153990915.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  get_wer = load_metric('wer')
/home/lee-j/github/supreme-parakeet/.venv/lib/python3.10/site-packages/datasets/load.py:756: FutureWarning: The repository for wer contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/wer/wer.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [9]:
import numpy as np

# function to compute wer
def compute_wer(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = get_wer.compute(predictions=pred_str, references=label_str)

    return {'wer' : wer}

## Model

Initialise the ```wav2vec2-large-960h``` from the HuggingFace checkpoint.
Loss reduction is set to mean to minimise GPU usage. Pad token is set to the pad token defined with the tokenizer.

In [10]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-960h", 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
)

Some weights of the model checkpoint at facebook/wav2vec2-large-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You s

As stated in https://arxiv.org/abs/2006.11477 , feature extraction is sufficiently pretrained and does not need further finetuning.

In [11]:
model.freeze_feature_extractor()

/home/lee-j/github/supreme-parakeet/.venv/lib/python3.10/site-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1920: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5. Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


## Training arguments

- ```group_by_length = True``` groups audio samples of similar lengths together into one batch to reduce the use of padding tokens.

In [2]:
# import pickle
# with open('dataset.pkl', 'rb') as f:
#     dataset = pickle.load(f)

/home/lee-j/github/supreme-parakeet/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
from transformers import TrainingArguments 

training_args = TrainingArguments(
    output_dir='./logs/',
    group_by_length=True,
    per_device_train_batch_size=32,
    evaluation_strategy="steps",
    num_train_epochs=30,
    fp16=True, 
    save_steps=500,
    eval_steps=500,
    logging_steps=500,
    learning_rate=1e-4,
    weight_decay=0.005,
    warmup_steps=1000,
    save_total_limit=2,
    )


## Trainer

The model, data collator, training arguments, metrics, feature extractor, and the data can be passed to the Trainer class for initialisation.

In [16]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_wer,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=processor.feature_extractor,
)


/home/lee-j/github/supreme-parakeet/.venv/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


## Training

In [17]:
trainer.train()

KeyboardInterrupt: 